# Deep Search integrations - Argilla.io

In this example we will use the output of the converted document for populating a dataset on
[Argilla](https://argilla.io). This enables the user to annotate text for multiple purposes,
e.g. text classification, named entities recognition, etc as well as train custom models fitting their purposes.


### Authentication via stored credentials

In this example, we initialize the Deep Search client from the credentials
contained in the file `../../ds-auth.json`. This can be generated with

```shell
!deepsearch login --output ../../ds-auth.json
```

More details in the [docs](https://ds4sd.github.io/deepsearch-toolkit/getting_started/#authentication).

### Setup your environment

In this example we require the connection to a running Argilla instance.

The [README](./README.md) file of this example describes in more details how to set it up.


### Notebooks parameters

The following block defines the parameters used to execute the notebook

- `CONFIG_FILE`: location of the Deep Search configuration file
- `INPUT_FILE`: the input PDF to converted and analyzed
- `ARGILLA_API_URL`: the API URL of the Argilla instance
- `ARGILLA_API_KEY`: the API Key of the Argilla instance
- `ARGILLA_DATASET`: the name of the dataset on Argilla
- `SPACY_MODEL`: the spaCy model to use for tokenization
    

In [1]:
# Input parameters for the example flow
import os
from pathlib import Path

CONFIG_FILE = Path("../../ds-auth.json")
PROJ_KEY = "1234567890abcdefghijklmnopqrstvwyz123456"

INPUT_FILE = Path("../../data/samples/2206.00785.pdf")

# Argilla configuration
ARGILLA_API_URL = os.getenv("ARGILLA_API_URL")
ARGILLA_API_KEY = os.getenv("ARGILLA_API_KEY")
ARGILLA_DATASET = "deepsearch-documents"
# Tokenization
SPACY_MODEL = "en_core_web_sm"

In [2]:
# Import standard dependenices
import json
import tempfile
import typing
from zipfile import ZipFile

# IPython utilities
from IPython.display import display, Markdown, HTML, display_html

# Import the deepsearch-toolkit
import deepsearch as ds

# Import specific to the example
import argilla as rg
import spacy
from pydantic import BaseModel

In [21]:
# Download the spaCy model
!python -m spacy download {SPACY_MODEL}

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 10.6 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
class DocTextSegment(BaseModel):
    page: int  # page number
    idx: int  # index of text segment in the document
    title: str  # title of the document
    name: str  # flavour of text segment
    type: str  # type of text segment
    text: str  # content of the text segment
    text_classification: typing.Any = (
        None  # this could be used to store predictions of text classification
    )
    token_classification: typing.Any = (
        None  # this could be used to store predictions of token classification
    )

## Document conversion with Deep Search

In [4]:
# Initialize the Deep Search client from the config file
config = ds.DeepSearchConfig.parse_file(CONFIG_FILE)
client = ds.CpsApiClient(config)
api = ds.CpsApi(client)

In [5]:
# Launch the docucment conversion and download the results
documents = ds.convert_documents(
    api=api, proj_key=PROJ_KEY, source_path=INPUT_FILE, progress_bar=True
)

Converting input:     : 100%|██████████████████████████████| 1/1 [00:23<00:00, 23.11s/it]                                                                                


In [6]:
output_dir = tempfile.mkdtemp()

documents.download_all(result_dir=output_dir, progress_bar=True)

converted_docs = {}
for output_file in Path(output_dir).rglob("json*.zip"):
    with ZipFile(output_file) as archive:
        all_files = archive.namelist()
        for name in all_files:
            if not name.endswith(".json"):
                continue

            doc_jsondata = json.loads(archive.read(name))
            converted_docs[f"{output_file}//{name}"] = doc_jsondata

print(f"{len(converted_docs)} documents have been loaded after conversion.")

1 have been loaded after conversion.


## Extract text segments

In [11]:
text_segments = []
for doc in converted_docs.values():

    doc_title = doc.get("description").get("title")
    for idx, text_segment in enumerate(doc["main-text"]):
        # filter only components with text
        if "text" not in text_segment:
            continue

        # append to the component to the list of segments
        text_segments.append(
            DocTextSegment(
                title=doc_title,
                page=text_segment.get("prov", [{}])[0].get("page"),
                idx=idx,
                name=text_segment.get("name"),
                type=text_segment.get("type"),
                text=text_segment.get("text"),
            )
        )

print(f"{len(text_segments)} text segments got extracted from the document")

133 got extracted from the document


## Log the text segments to Argilla

In [14]:
# Initialize the Argilla SDK
rg.init(api_url=ARGILLA_API_URL, api_key=ARGILLA_API_KEY)

# Initialize the spaCy NLP model for the tokenization of the text
nlp = spacy.load("en_core_web_sm")

In [15]:
# Prepare text segments for text classification

records_text_classificaiton = []
for segment in text_segments:
    records_text_classificaiton.append(
        rg.TextClassificationRecord(
            text=segment.text,
            vectors={},
            prediction=segment.text_classification,
            metadata=segment.dict(
                exclude={"text", "text_classification", "token_classification"}
            ),
        )
    )

In [16]:
# Submit text for classification
rg.log(records_text_classificaiton, name=f"{ARGILLA_DATASET}-text")

Output()

133 records logged to ]8;id=303262;https://dolfim-ibm-argilla-test.hf.space/datasets/admin/deepsearch-documents-text\https://dolfim-ibm-argilla-test.hf.space/datasets/admin/deepsearch-documents-text]8;;\

BulkResponse(dataset='deepsearch-documents-text', processed=133, failed=0)

In [17]:
# Prepare text segments for token classification

records_token_classificaiton = []
for segment in text_segments:
    records_token_classificaiton.append(
        rg.TokenClassificationRecord(
            text=segment.text,
            tokens=[token.text for token in nlp(segment.text)],
            prediction=segment.token_classification,
            vectors={},
            metadata=segment.dict(
                exclude={"text", "text_classification", "token_classification"}
            ),
        )
    )

In [18]:
# Submit tokens for classification
rg.log(records_token_classificaiton, name=f"{ARGILLA_DATASET}-token")

Output()

133 records logged to ]8;id=556751;https://dolfim-ibm-argilla-test.hf.space/datasets/admin/deepsearch-documents-token\https://dolfim-ibm-argilla-test.hf.space/datasets/admin/deepsearch-documents-token]8;;\

BulkResponse(dataset='deepsearch-documents-token', processed=133, failed=0)

## What's next?

Now that the documents are converted and uploaded in Argilla, you can use the links printed above to annotate and train your own models.

Visit the <a href="https://docs.argilla.io" rel="nofollow" target="_blank">Argilla documentation</a> to learn about its features and check out the <a href="https://docs.argilla.io/en/latest/guides/guides.html" rel="nofollow" target="_blank">Deep Dive Guides</a> and <a href="https://docs.argilla.io/en/latest/tutorials/tutorials.html" rel="nofollow" target="_blank">Tutorials</a>.
